# Pagpapatupad ng isang Animal Expert System

Isang halimbawa mula sa [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

Sa halimbawang ito, magpapatupad tayo ng isang simpleng knowledge-based system upang matukoy ang isang hayop batay sa ilang pisikal na katangian. Ang sistema ay maaaring ilarawan sa pamamagitan ng sumusunod na AND-OR tree (ito ay bahagi ng buong puno, madali tayong makakapagdagdag ng iba pang mga patakaran):

![](../../../../../../translated_images/tl/AND-OR-Tree.5592d2c70187f283.webp)


## Ang aming sariling expert systems shell na may backward inference

Subukan nating magdefine ng isang simpleng wika para sa knowledge representation batay sa production rules. Gagamit tayo ng mga Python classes bilang mga keyword para magdefine ng mga rules. Mayroong tatlong pangunahing uri ng mga klase:
* Ang `Ask` ay kumakatawan sa isang tanong na kailangang itanong sa gumagamit. Naglalaman ito ng set ng mga posibleng sagot.
* Ang `If` ay kumakatawan sa isang patakaran, at ito ay isang syntactic sugar lamang para itago ang nilalaman ng patakaran
* Ang `AND`/`OR` ay mga klase na kumakatawan sa mga sanga ng AND/OR ng puno. Tinatago lamang nila ang listahan ng mga argumento sa loob. Upang mapadali ang kodigo, lahat ng functionality ay dinefine sa parent class na `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Sa aming sistema, ang working memory ay naglalaman ng listahan ng **mga katotohanan** bilang **mga pares ng katangian-halaga**. Ang knowledgebase ay maaaring tukuyin bilang isang malaking diksyunaryo na nagmamapa ng mga aksyon (mga bagong katotohanang dapat ipasok sa working memory) sa mga kondisyon, na ipinapahayag bilang AND-OR na mga ekspresyon. Gayundin, ang ilang mga katotohanan ay maaaring `Ask`-in.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Upang isagawa ang backward inference, magtatakda tayo ng klase na `Knowledgebase`. Ito ay maglalaman ng:
* Gumaganang `memory` - isang diksyunaryo na nagmamapa ng mga katangian sa mga halaga
* Mga patakaran ng Knowledgebase sa format na inilalarawan sa itaas

Dalawang pangunahing mga pamamaraan ay:
* `get` upang kunin ang halaga ng isang katangian, isinasagawa ang inference kung kinakailangan. Halimbawa, ang `get('color')` ay kukuha ng halaga ng slot na kulay (tatanungin kung kinakailangan, at itatago ang halaga para sa susunod na paggamit sa working memory). Kung hihingin natin ang `get('color:blue')`, tatanungin nito ang kulay, at ibabalik ang halagang `y`/`n` depende sa kulay.
* `eval` ang nagsasagawa ng aktwal na inference, ibig sabihin ay naglalakbay sa AND/OR na puno, sinusuri ang mga sub-goals, at iba pa.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Ngayon ay tukuyin natin ang ating kaalaman tungkol sa mga hayop at isagawa ang konsultasyon. Tandaan na ang tawag na ito ay magtatanong sa iyo. Maaari kang sumagot sa pamamagitan ng pag-type ng `y`/`n` para sa mga tanong na oo-hindi, o sa pamamagitan ng pagtukoy ng numero (0..N) para sa mga tanong na may mas mahahabang sagot na multiple-choice.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Paggamit ng Experta para sa Forward Inference

Sa susunod na halimbawa, susubukan nating ipatupad ang forward inference gamit ang isa sa mga library para sa knowledge representation, ang [Experta](https://github.com/nilp0inter/experta). **Ang Experta** ay isang library para sa paglikha ng mga forward inference system sa Python, na disenyo upang maging katulad ng klasikong lumang system na [CLIPS](http://www.clipsrules.net/index.html).

Maaari rin sana namin ipatupad ang forward chaining nang mag-isa nang walang masyadong problema, ngunit ang mga payak na implementasyon ay karaniwang hindi gaanong epektibo. Para sa mas epektibong pag-match ng mga patakaran, isang espesyal na algorithm na [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) ang ginagamit.


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Ide-define natin ang ating sistema bilang isang klase na nag-subclass sa `KnowledgeEngine`. Bawat patakaran ay dine-define ng isang hiwalay na function na may anotasyong `@Rule`, na nagsasaad kung kailan dapat mag-fire ang patakaran. Sa loob ng patakaran, maaari tayong magdagdag ng mga bagong katotohanan gamit ang function na `declare`, at ang pagdagdag ng mga katotohanang iyon ay magreresulta sa pagtawag ng iba pang mga patakaran ng forward inference engine.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Kapag naitakda na natin ang isang knowledgebase, pinupuno natin ang ating working memory ng ilang paunang mga katotohanan, at pagkatapos ay tinatawag ang `run()` na metodo upang isagawa ang inference. Makikita mo bilang resulta na ang mga bagong napatunayang katotohanan ay idinaragdag sa working memory, kabilang ang panghuling katotohanan tungkol sa hayop (kung tama nating naitakda ang lahat ng paunang mga katotohanan).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Paalala**:
Ang dokumentong ito ay isinalin gamit ang serbisyo ng AI na pagsasalin na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagaman aming pinagsisikapang maging tumpak ang pagsasalin, pakatandaan na ang awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na mapagkakatiwalaang sanggunian. Para sa mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na nagmumula sa paggamit ng pagsasaling ito.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
